# 1. Get NL-Augmenter and install dependencies

## Copy git repository of NL-Augmenter

In [1]:
!git clone https://www.github.com/GEM-benchmark/NL-Augmenter

Cloning into 'NL-Augmenter'...
remote: Enumerating objects: 11949, done.
remote: Counting objects: 100% (620/620), done.
remote: Compressing objects: 100% (414/414), done.
remote: Total 11949 (delta 349), reused 384 (delta 205), pack-reused 11329
Receiving objects: 100% (11949/11949), 91.25 MiB | 21.68 MiB/s, done.
Resolving deltas: 100% (7799/7799), done.


## Go to newly copied directory

In [2]:
cd NL-Augmenter

/content/NL-Augmenter


## Install requirements

In [3]:
!pip install -r requirements.txt --quiet

     |████████████████████████████████| 12.1 MB 29.3 MB/s 
     |████████████████████████████████| 12.7 MB 194 kB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 280 kB 27.3 MB/s 
     |████████████████████████████████| 234 kB 60.4 MB/s 
     |████████████████████████████████| 2.2 MB 40.6 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.1 MB/s 
     |████████████████████████████████| 138 kB 72.0 MB/s 
     |████████████████████████████████| 190 kB 45.6 MB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 105 kB 58.9 MB/s 
     |████████████████████████████████| 22.3 MB 14 kB/s 
     |████████████████████████████████| 167 kB 62.4 MB/s 
     |████████████████████████████████| 9.1 MB 45.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████

# 2. Load the data

##  Get dataloader and use it to load OrangeSUM dataset

In [11]:
from datasets import load_dataset

data = load_dataset("orange_sum", 'abstract')

Dataset gem downloaded and prepared to /root/.cache/huggingface/datasets/gem/common_gen/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4. Subsequent calls will reuse this data.


## Explore data structure

In [99]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 21401
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 1500
    })
})

## Retrieve one entry from a train subset

In [100]:
train_instance = data['train'][0]
train_instance

{'summary': "L'information n'a pas été confirmée par l'intéressé qui déclare toutefois étudier la question.\n",
 'text': 'Thierry Mariani sur la liste du Rassemblement national (RN, ex-FN) aux européennes ? C\'est ce qu\'affirme mardi 11 septembre Chez Pol, la nouvelle newsletter politique de Libération. L\'ancien député Les Républicain et ministre de Nicolas Sarkozy serait sur le point de rejoindre les troupes de Marine Le Pen pour le élections européennes de 2019. "Ça va se faire. Ce n\'est plus qu\'une question de calendrier. On n\'est pas obligé de l\'annoncer tout de suite, à huit mois des européennes", aurait ainsi assuré un membre influent du RN. Contacté par Franceinfo, M. Mariani n\'a pas confirmé l\'information. "Les élections sont en juin, je ne sais même pas qui sera numéro 1 sur la liste", a répondu l\'ancien ministre des Transports. Il reconnaît toutefois, toujours cité par Franceinfo, que son nom sur la liste du RN "fait partie des possibilités". "Fréjus est une ville sy

In [101]:
def inject_gem_id(dataset_object, split_name):
    split_with_id = []
    for i in range(len(dataset_object[split_name])):
        my_object = {
            'gem_id': 'orange_sum-' + split_name + '-' + str(i),
            'summary': dataset_object[split_name][i].get("summary"), 
            'text': dataset_object[split_name][i].get("text") 
        }
        split_with_id.append(my_object)
    return split_with_id

train_split = inject_gem_id(data, "train")
print("Train split length", len(train_split))

validation_split = inject_gem_id(data, "validation")
print("Validation split length", len(validation_split))

test_split = inject_gem_id(data, "test")
print("Test split length", len(test_split))

Train split length 21401
Validation split length 1500
Test split length 1500


In [102]:
train_split[0]

{'gem_id': 'orange_sum-train-0',
 'summary': "L'information n'a pas été confirmée par l'intéressé qui déclare toutefois étudier la question.\n",
 'text': 'Thierry Mariani sur la liste du Rassemblement national (RN, ex-FN) aux européennes ? C\'est ce qu\'affirme mardi 11 septembre Chez Pol, la nouvelle newsletter politique de Libération. L\'ancien député Les Républicain et ministre de Nicolas Sarkozy serait sur le point de rejoindre les troupes de Marine Le Pen pour le élections européennes de 2019. "Ça va se faire. Ce n\'est plus qu\'une question de calendrier. On n\'est pas obligé de l\'annoncer tout de suite, à huit mois des européennes", aurait ainsi assuré un membre influent du RN. Contacté par Franceinfo, M. Mariani n\'a pas confirmé l\'information. "Les élections sont en juin, je ne sais même pas qui sera numéro 1 sur la liste", a répondu l\'ancien ministre des Transports. Il reconnaît toutefois, toujours cité par Franceinfo, que son nom sur la liste du RN "fait partie des possib

In [103]:
validation_split[0]

{'gem_id': 'orange_sum-validation-0',
 'summary': 'Le bateau de croisière, long de 275 m, a percuté un quai lors de son arrivée dans le port de Venise, dimanche 2 juin. Quatre personnes ont été blessées.\n',
 'text': "Sur les réseaux sociaux, les images sont impressionnantes. Dimanche matin à Venise, l'équipage du MSC Opéra a perdu le contrôle du paquebot, à son arrivée dans le port de la cité des Doges. Le navire, qui peut contenir plus de 2.600 passagers, est venu heurter le quai auquel il voulait s'arrimer. Le paquebot a raclé le quai sur plusieurs mètres, suscitant la panique des personnes à terre, avant de percuter un autre bateau touristique, le Michelangelo, stoppant ainsi sa course. Des témoins ont filmé la scène. Les vidéos montrent des touristes courant pour tenter de fuir le paquebot, qui ne semble pas vouloir s'arrêter. Quatre personnes ont été blessées dans cet accident : deux légèrement, tandis que les deux autres ont été transportées à l'hôpital pour des examens. L'incid

In [104]:
test_split[0]

{'gem_id': 'orange_sum-test-0',
 'summary': "En janvier dernier, l'animateur de D8 avait versé des nouilles dans le pantalon de son chroniqueur Matthieu Delormeau. Une séquence très commentée à l'époque.\n",
 'text': 'Une scène "sans précédent dans l\'histoire de l\'abjection à la télévision". C\'est ainsi que Bruno Donnet, journaliste sur France Inter, qualifiait il y a quelques mois une séquence diffusée fin janvier dans "Touche pas à mon poste" sur D8. On y voit Cyril Hanouna et Stéphane Rotenberg, le présentateur de "Top Chef" sur M6, verser un bol de nouilles dans le slip du chroniqueur Matthieu Delormeau en guise de gage. Pour le journaliste de la radio publique, "cette séquence (...) normalise un comportement qui ne l\'est pas. Humilier quelqu\'un, le harceler au travail, c\'est interdit par la loi", s\'était insurgé Bruno Donnet après la diffusion de cette séquence. "Ce qu\'il est important de retenir ici, c\'est qu\'il (Matthieu Delormeau) dit non. Il exprime son refus, avait 

## Retrieve a target sentence from this entry object

In [105]:
sentence = train_instance.get("summary")
sentence

"L'information n'a pas été confirmée par l'intéressé qui déclare toutefois étudier la question.\n"

# 3. Test the filter

## Load a filter class from a corresponding ditectory and try the example from its ReadMe

In [106]:
from filters.gender_bias import GenderBiasFilter

f = GenderBiasFilter("fr")

## Take a subset of target sentences from OrangeSUM dataset 


In [107]:
def get_sentences(split, num_iter):
    sentences = []
    for i in range(num_iter):
        sentence = split[i].get("summary")
        sentences.append(sentence)
    return sentences

sentences = get_sentences(train_split, 10)
sentences

["L'information n'a pas été confirmée par l'intéressé qui déclare toutefois étudier la question.\n",
 "Le maire de Bordeaux ne fait plus partie des Républicains et il tient à le montrer. Lors de ses voeux à la presse, l'édile a pris ses distances avec sa famille politique historique, qu'il trouve trop proche de Marine Le Pen.\n",
 'En 2020, les tribunaux d\'instance fusionnent avec ceux de grande instance pour former un unique "tribunal judiciaire". C\'est la principale mesure de la réforme de la Justice, portée par la garde des sceaux Nicole Belloubet.\n',
 'Les médecins jugés "gros prescripteurs d\'arrêts maladie", sont notamment pointés du doigt par le patron du Medef Geoffroy Roux de Bézieux dans une interview au Figaro.\n',
 'Il aura fallu mobiliser 90 gendarmes pour cette opération d\'envergure menée en Gironde, dans le Var et dans les Pyrénées-Orientales mardi. Dix personnes ont été arrêtées, soupçonnées d\'avoir participé à une escroquerie "d\'ampleur" sur des voitures de luxe.

## Apply a filter to this small subset

In [108]:
f.filter(sentences)

False

## Explore statistics which makes a filter returning the presence of gender bias

In [109]:
def get_stats(text, language):
    flagged_sentences = f.flag_sentences(text, language)
    feminine, masculine, neutral = f.count_genders(flagged_sentences)
    
    print("Feminine flagged sentences:", feminine)
    print("Masculine flagged sentences:", masculine)
    print("Neutral flagged sentences:", neutral)

get_stats(sentences, "fr")

Feminine flagged sentences: 2
Masculine flagged sentences: 1
Neutral flagged sentences: 7


## Retrieve the sentences, corresponding to the statistics

In [110]:
def get_phrases(text, language):
    flagged_sentences = f.flag_sentences(text, language)
    feminine_group, masculine_group, neutral_group = f.sort_groups(flagged_sentences)    
    return feminine_group, masculine_group, neutral_group

feminine_group, masculine_group, neutral_group = get_phrases(sentences, "fr")

print("This is a feminine group:", feminine_group)
print("This is a masculine group:", masculine_group)
print("This is a neutral group:", neutral_group)

This is a feminine group: ['En 2020, les tribunaux d\'instance fusionnent avec ceux de grande instance pour former un unique "tribunal judiciaire". C\'est la principale mesure de la réforme de la Justice, portée par la garde des sceaux Nicole Belloubet.\n', "Marine Le Pen a prévu de se rendre sur l'Île-de-Sein ce jeudi pour célébrer l'appel du 18 juin. Le maire a déploré que la venue de la présidente du Rassemblement national perturbe la commémoration officielle.\n"]
This is a masculine group: ['Il aura fallu mobiliser 90 gendarmes pour cette opération d\'envergure menée en Gironde, dans le Var et dans les Pyrénées-Orientales mardi. Dix personnes ont été arrêtées, soupçonnées d\'avoir participé à une escroquerie "d\'ampleur" sur des voitures de luxe.\n']
This is a neutral group: ["L'information n'a pas été confirmée par l'intéressé qui déclare toutefois étudier la question.\n", "Le maire de Bordeaux ne fait plus partie des Républicains et il tient à le montrer. Lors de ses voeux à la p

In [111]:
def create_filtered(split, num_iter, feminine_group, masculine_group, neutral_group):
    female = []
    male = []
    neutral = []

    for i in range(num_iter):
        train_instance = split[i].get("summary")
        
        if train_instance in feminine_group:
            female.append(split[i].get("gem_id"))

        elif train_instance in masculine_group:
            male.append(split[i].get("gem_id"))

        elif train_instance in neutral_group:
            neutral.append(split[i].get("gem_id"))

    filtered_entries = {
        "female": female,
        "male": male,
        "neutral": neutral
    }

    filtered_object = {
        "gender_bias": filtered_entries
    }

    return filtered_object

## Create final object with filtered data

In [112]:
filtered_object = create_filtered(train_split, 10, feminine_group, masculine_group, neutral_group)
filtered_object

{'gender_bias': {'female': ['orange_sum-train-2', 'orange_sum-train-6'],
  'male': ['orange_sum-train-4'],
  'neutral': ['orange_sum-train-0',
   'orange_sum-train-1',
   'orange_sum-train-3',
   'orange_sum-train-5',
   'orange_sum-train-7',
   'orange_sum-train-8',
   'orange_sum-train-9']}}

## Store the object in a json file

In [84]:
import json

with open('orange_sum_contrast_sets-test.json', 'w') as outfile:
    json.dump(filtered_object, outfile)

# 4. Apply the process to multiple entries

In [113]:
#Train
sentences = get_sentences(train_split, len(train_split))
f.filter(sentences)

False

In [114]:
get_stats(sentences, "fr")

Feminine flagged sentences: 4359
Masculine flagged sentences: 4339
Neutral flagged sentences: 12703


In [115]:
feminine_group, masculine_group, neutral_group = get_phrases(sentences, "fr")
filtered_object_train = create_filtered(train_split, len(train_split), feminine_group, masculine_group, neutral_group)

In [116]:
#Validation
sentences = get_sentences(validation_split, len(validation_split))
f.filter(sentences)

False

In [117]:
get_stats(sentences, "fr")

Feminine flagged sentences: 304
Masculine flagged sentences: 287
Neutral flagged sentences: 909


In [118]:
feminine_group, masculine_group, neutral_group = get_phrases(sentences, "fr")
filtered_object_validation = create_filtered(validation_split, len(validation_split), feminine_group, masculine_group, neutral_group)

In [119]:
#Test
sentences = get_sentences(test_split, len(test_split))
f.filter(sentences)

True

In [120]:
get_stats(sentences, "fr")

Feminine flagged sentences: 290
Masculine flagged sentences: 300
Neutral flagged sentences: 910


In [121]:
feminine_group, masculine_group, neutral_group = get_phrases(sentences, "fr")
filtered_object_test = create_filtered(test_split, len(test_split), feminine_group, masculine_group, neutral_group)

In [122]:
filtered_object = {
    'train': filtered_object_train,
    'validation': filtered_object_validation,
    'test': filtered_object_test
}

In [123]:
with open('orange_sum_contrast_sets.json', 'w') as outfile:
    json.dump(filtered_object, outfile)